# 0. Import Data (v.1)

In [1]:
# HC - Hardcoded; not ideal

import pandas as pd
from sklearn.model_selection import train_test_split
import glob
import os
import numpy

wave_path = r'C:\Users\olusa\Documents\VS Code\ML\MVP\Gestures\Wave' # HC
all_files_in_wave = glob.glob(os.path.join(wave_path, "*.csv"))

wave_X = numpy.zeros((len(all_files_in_wave),200,6)) # HC num examples x num samples per example x num inputs

csvIndex = 0
for csv in all_files_in_wave:
    wave_X[csvIndex] = pd.read_csv(csv)
    csvIndex += 1

In [2]:
noise_path = r'C:\Users\olusa\Documents\VS Code\ML\MVP\Gestures\Noise' # HC
all_files_in_noise = glob.glob(os.path.join(noise_path, "*.csv"))

noise_X = numpy.zeros((len(all_files_in_noise),200,6)) # HC

csvIndex = 0
for csv in all_files_in_noise:
    noise_X[csvIndex] = pd.read_csv(csv)
    csvIndex += 1

In [3]:
X = numpy.concatenate([noise_X, wave_X]) # HC
# print (X.shape)

In [4]:
wave_y = numpy.zeros(len(all_files_in_wave)) # HC
noise_y = numpy.ones(len(all_files_in_noise)) # HC

y = numpy.concatenate([noise_y, wave_y]) # HC

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [6]:
print(X_train.shape, X_test.shape)
print(y_train)

(16, 200, 6) (4, 200, 6)
[1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1.]


# 0. Import Data (v.2)

In [7]:
# HC - Hardcoded; not ideal

import pandas as pd
from sklearn.model_selection import train_test_split
import glob
import os
import numpy

# Sample config vars
num_readings_per_sample = 200
num_values_per_reading = 6

# Wave dataframe import
wave_df = pd.read_csv('./Gestures/Wave2/waves.csv')
# wave_df.head()
# print(len(wave_df))

# Separating samples into 3D Matrix, wave_X
num_samples_in_wave_df = int(len(wave_df) / num_readings_per_sample)

## Init wave_X
wave_X = numpy.zeros((num_samples_in_wave_df, num_readings_per_sample, num_values_per_reading))

layerIndex = 0
for sampleLayer in wave_X:
    first_row_index = layerIndex * num_readings_per_sample
    last_row_index = (layerIndex * num_readings_per_sample) + num_readings_per_sample
    wave_X[layerIndex] = wave_df.iloc[first_row_index:last_row_index, :]
    layerIndex += 1

print(wave_X.shape)

(20, 200, 6)


In [8]:
X = numpy.concatenate([wave_X]) # HC
# print (X.shape)

In [9]:
wave_y = numpy.ones(num_samples_in_wave_df) # HC

y = numpy.concatenate([wave_y]) # HC

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [11]:
print(X_train.shape, X_test.shape)
print(y_train)

(16, 200, 6) (4, 200, 6)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


# 1. Import Dependencies

In [12]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten
from sklearn.metrics import accuracy_score

# 2. Build and Compile Model

In [13]:
model = Sequential()

model.add(Flatten(input_shape=(num_readings_per_sample, num_values_per_reading)))
model.add(Dense(units=1, activation='sigmoid'))

In [14]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

# 3. Fit, Predict and Evaluate

In [15]:
model.fit(X_train, y_train, epochs=4, batch_size=1)

Epoch 1/4
16/16 [==============================] - 0s 1ms/step - loss: 10.2170 - accuracy: 0.8125
Epoch 2/4
16/16 [==============================] - 0s 2ms/step - loss: 2.7391 - accuracy: 0.9375
Epoch 3/4
16/16 [==============================] - 0s 2ms/step - loss: 3.5754e-15 - accuracy: 1.0000
Epoch 4/4
16/16 [==============================] - 0s 1ms/step - loss: 3.5754e-15 - accuracy: 1.0000


In [16]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_quant_model = converter.convert()

In [17]:
y_hat = model.predict(X_test)
y_hat = [0 if val <0.5 else 1 for val in y_hat]

In [18]:
accuracy_score(y_test, y_hat)

1.0

# 4. Save and Convert to TFLite Model

In [19]:
model.save('model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [20]:
open("converted_model.tflite", "wb").write(tflite_quant_model)

2432

# Test Code

In [21]:
from tensorflow import keras
# from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
#save the model architecture to JSON file
with open('model.json', 'w') as json_file:
    json_file.write(json_model)
#saving the weights of the model
model.save_weights('model_weights.h5')